In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader,TensorDataset
import torch.utils.data as data
import numpy as np
import scipy.io as sio
import math
import argparse
import random
import os
from My_Loss import HardTripletLoss
from My_Loss import HardTripletLoss2
from My_Loss import HardTripletLoss_D
from tensorboardX import SummaryWriter
import visdom

In [2]:
BATCH_SIZE = 64
EPISODE = 100000
TEST_EPISODE = 1000
LEARNING_RATE =1e-5
Weight_Deacy = 0
GPU = 0
Margin = 3

In [3]:
print("init dataset")
##################################参数##################################################################
dataroot = '../data'
dataset = 'AwA1_data'
image_embedding = 'res101'               #ResNet101层
class_embedding = 'original_att'         #属性表达 85-d
#######################################读取视觉特征###################################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + image_embedding + ".mat")  #scipy loadmat
 
feature = matcontent['features'].T         #转置 30478x2048 每一行是一个完整的样本

label = matcontent['labels'].astype(int).squeeze() - 1   #matlab begin 1 ,numpy begin 0
########################################读取属性特征###########################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + class_embedding + "_splits.mat")

    
# numpy array index starts from 0, matlab starts from 1
trainval_loc = matcontent['trainval_loc'].squeeze() - 1    #squeeze()去掉维度中的1 AxBx1 --->AxB

test_seen_loc = matcontent['test_seen_loc'].squeeze() - 1
test_unseen_loc = matcontent['test_unseen_loc'].squeeze() - 1

attribute = matcontent['att'].T    #转置 50x85 每行是整个属性向量

x = feature[trainval_loc]                      # train_features trainval里面是图片的编号 begin with 0 ，19832个
train_label = label[trainval_loc].astype(int)  # train_label  int类型没变 每个图片的lable 19832个
train_id = np.unique(train_label)

att = attribute[train_label]                   # train attributes 每个图片的属性 19832个

    
x_test = feature[test_unseen_loc]                   # test_feature 5685个
test_label = label[test_unseen_loc].astype(int)     # test_label   5685个

x_test_seen = feature[test_seen_loc]                #test_seen_feature 4958个
test_label_seen = label[test_seen_loc].astype(int)  # test_seen_label  4958个
    
test_id = np.unique(test_label)                     # test_id  10个类 ，unique去重
att_pro = attribute[test_id]                        # test_attribute 每一类的属性向量 10x85

# train set
train_features = torch.from_numpy(x)   #np-->tensor
#train_fearures_negative = torch.from_numpy(x_negative)

sample_attributes=[]
train_label = torch.from_numpy(train_label).unsqueeze(1) #每张图片的属性转化 ，unsqueeze(1)就是插入到第一维度 AxB维-->Ax1xB
# attributes
all_attributes = np.array(attribute)  #所有50类属性转变为numpy数组???属性向量仍然用的numpy类型 没有转化为pytorch
    
#print('-'*50)
attributes = torch.from_numpy(attribute) 
#print(attribute)
# test set

test_features = torch.from_numpy(x_test)
#print(test_features.shape)

test_label = torch.from_numpy(test_label).unsqueeze(1)
#print(test_label.shape)

testclasses_id = np.array(test_id)
#print(testclasses_id.shape)

test_attributes = torch.from_numpy(att_pro).float()
#print(test_attributes.shape)

test_seen_features = torch.from_numpy(x_test_seen)
#print(test_seen_features.shape)

test_seen_label = torch.from_numpy(test_label_seen)

train_data = TensorDataset( train_label, train_features )
#train_data = TensorDataset(train_label, train_features, train_fearures_negative)

#################here need new code to make triplet data#####################
print('-'*100)

init dataset
----------------------------------------------------------------------------------------------------


In [4]:
from my_net_3 import AttributeNetwork
from my_net_3 import MetricNetwork

In [5]:
# init network
print("init networks")
attribute_network = AttributeNetwork(85,1024,2048)  #85d属性 1024隐藏层 2048输出 85d到2048d
metric_network = MetricNetwork(2048,1024,2048)

attribute_network.cuda(GPU) 
metric_network.cuda(GPU)

#学习率每200k步 乘0.5
attribute_network_optim = torch.optim.Adam(attribute_network.parameters(), lr=LEARNING_RATE,weight_decay=Weight_Deacy)
metric_network_optim = torch.optim.Adam(metric_network.parameters(), lr=LEARNING_RATE,weight_decay=Weight_Deacy)
#
#triplet_network_optim = torch.optim.SGD(triplet_network.parameters(), lr=LEARNING_RATE,momentum=0.9 , 
#                                         weight_decay=Weight_Deacy)

attribute_network_scheduler = StepLR(attribute_network_optim , step_size=20000 , gamma=0.5)
metric_network_scheduler = StepLR(metric_network_optim , step_size=20000 , gamma=0.5)
#
print('-'*100)

init networks
----------------------------------------------------------------------------------------------------


In [6]:
def compute_accuracy(test_features, test_label, test_id, test_attributes):
    
    test_data = TensorDataset(test_features, test_label)
    test_batch = 32
    test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False)
    total_rewards = 0

    sample_labels = test_id
    sample_attributes = test_attributes
    class_num = sample_attributes.shape[0]
    test_size = test_features.shape[0]

    print("class num:", class_num)

    for batch_features,batch_labels in test_loader:

        batch_size = batch_labels.shape[0]
        batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(class_num, 0))
        batch_features_ext = metric_network(Variable(batch_features_ext).cuda(GPU).float())  # 32*1024


        sample_features = metric_network(attribute_network(Variable(sample_attributes).cuda(GPU).float()))
        sample_features_ext = sample_features.repeat(batch_size, 1)


        relations = F.pairwise_distance(batch_features_ext, sample_features_ext, 2).view(-1, class_num)
        re_batch_labels = []
        for label in batch_labels.numpy():
            index = np.argwhere(sample_labels == label)
            re_batch_labels.append(index[0][0])
        re_batch_labels = torch.cuda.LongTensor(re_batch_labels)


        _, predict_labels = torch.min(relations.data, 1)
        rewards = [1 if predict_labels[j] == re_batch_labels[j] else 0 for j in range(batch_size)]
        total_rewards += np.sum(rewards)
    test_accuracy = total_rewards/1.0/test_size
    return  test_accuracy

In [7]:
def compute_accuracy_per_class(test_features, test_label, test_id, test_attributes):
    
    test_data = TensorDataset(test_features, test_label)
    test_batch = 32
    test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False)
    total_rewards = 0
    #print(test_features.size())
    sample_labels = test_id
    sample_attributes = test_attributes
    class_num = sample_attributes.shape[0]
    test_size = test_features.shape[0]
    per_class_num = np.zeros(class_num)
    per_class_true= np.zeros(class_num)

    print("class num:", class_num)

    for batch_features,batch_labels in test_loader:

        batch_size = batch_labels.shape[0]
        batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(class_num, 0))
        batch_features_ext = metric_network(Variable(batch_features_ext).cuda(GPU).float())  # 32*1024


        sample_features = metric_network(attribute_network(Variable(sample_attributes).cuda(GPU).float()))
        sample_features_ext = sample_features.repeat(batch_size, 1)


        relations = F.pairwise_distance(batch_features_ext, sample_features_ext, 2).view(-1, class_num)
        re_batch_labels = []
        for label in batch_labels.numpy():
            index = np.argwhere(sample_labels == label)
            re_batch_labels.append(index[0][0])
        re_batch_labels_id, batch_per_num = np.unique(re_batch_labels , return_counts=True) 
        re_batch_labels = torch.cuda.LongTensor(re_batch_labels)
        
        for each in range(re_batch_labels_id.size):
            #print(re_batch_labels_id[each])
            #print(batch_per_num[each])
            per_class_num[re_batch_labels_id[each]] = per_class_num[re_batch_labels_id[each]] + batch_per_num[each]
        #print(re_batch_labels_id)
        #print('-'*100)
        #print(batch_per_num)
        #print('-'*100)


        _, predict_labels = torch.min(relations.data, 1)
        for j in range(batch_size):
            if predict_labels[j] == re_batch_labels[j]:
                per_class_true[re_batch_labels[j]] = per_class_true[re_batch_labels[j]] + 1
            
    per_accuracy = per_class_true[np.nonzero(per_class_num)] / per_class_num[np.nonzero(per_class_num)]
    

    test_accuracy = np.sum(per_accuracy)/1.0/np.count_nonzero(per_class_num)
   
    #print(np.count_nonzero(per_class_num))
    return  test_accuracy

In [8]:
print("training...")
last_accuracy = 0.0
loss_zero_num =0
#writer = SummaryWriter()
#vis = visdom.Visdom()
for episode in range(EPISODE):

    attribute_network_scheduler.step(episode)
    metric_network_scheduler.step(episode)

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

    batch_labels, batch_features = train_loader.__iter__().next()
    batch_id = np.unique(batch_labels)

    batch_attributes = torch.Tensor([all_attributes[i] for i in batch_id]).squeeze(1)
    batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(batch_id.size, 0))
    batch_attributes_ext = batch_attributes.repeat(BATCH_SIZE, 1)

    batch_features_ext = Variable(batch_features_ext).cuda(GPU).float()  # 32*2048
    batch_attributes_ext = Variable(batch_attributes_ext).cuda(GPU)


    re_batch_labels = []
    for label in batch_labels.numpy():
        index = np.argwhere(batch_id == label)
        re_batch_labels.append(index[0][0])
    re_batch_labels = torch.cuda.LongTensor(re_batch_labels)
    re_batch_labels = Variable(re_batch_labels).cuda(GPU)
    

    criterion = HardTripletLoss(margin = Margin).cuda(GPU)
    triplet_loss= criterion(metric_network(attribute_network(batch_attributes_ext)), 
                            metric_network(batch_features_ext), re_batch_labels)
    if triplet_loss == 0:
        loss_zero_num = loss_zero_num + 1
    
    
    attribute_network.zero_grad()
    metric_network.zero_grad()
    
    triplet_loss.backward()
    
    attribute_network_optim.step()
    metric_network_optim.step()

    
    if (episode+1)%200 == 0 or episode==0:
        print("episode:", episode+1, "loss", triplet_loss)
        print('loss_zero_number= ',loss_zero_num)
        loss_zero_num = 0
        #writer.add_scalar('data/loss', triplet_loss, episode)
    
    if (episode+1)%200 == 0 or episode==0:
        print("Testing...")
        #attribute_network.eval()
        zsl_accuracy = compute_accuracy_per_class(test_features, test_label, test_id, test_attributes)
        gzsl_unseen_accuracy = compute_accuracy_per_class(test_features, test_label, np.arange(50), attributes)
        gzsl_seen_accuracy = compute_accuracy_per_class(test_seen_features, test_seen_label, np.arange(50), attributes)
        H = 2 * gzsl_seen_accuracy * gzsl_unseen_accuracy / (gzsl_unseen_accuracy + gzsl_seen_accuracy)
        #H2 = 2 * gzsl_seen_accuracy2 * gzsl_unseen_accuracy2 / (gzsl_unseen_accuracy2 + gzsl_seen_accuracy2)
        print('zsl:', zsl_accuracy)
        #print('zsl:', zsl_accuracy2)
        print('gzsl: unseen=%.4f , seen=%.4f , h=%.4f' % (gzsl_unseen_accuracy , gzsl_seen_accuracy, H))
        #print('gzsl: unseen=%.4f , seen=%.4f , h=%.4f' % (gzsl_unseen_accuracy2 , gzsl_seen_accuracy2, H2))
        print('_'*100)
        #writer.add_scalar('data/zsl_accuracy', zsl_accuracy, episode)
        #writer.add_scalar('data/gzsl_unseen_accuracy', gzsl_unseen_accuracy, episode)
        #writer.add_scalar('data/gzsl_seen_accuracy', gzsl_seen_accuracy, episode)
        #writer.add_scalar('data/H', H, episode)
#features = images.view(, 784)
#print(vis_label.size())
#writer.add_embedding(vis_images, metadata = vis_label.tolist())
#writer.add_embedding(vis_att, metadata = vis_label_att)
#writer.export_scalars_to_json("./test.json")
#writer.close()

training...
episode: 1 loss tensor(3.2020, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.14996092534229755
gzsl: unseen=0.0000 , seen=0.0103 , h=0.0000
____________________________________________________________________________________________________
episode: 200 loss tensor(2.9739, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.3249370155778787
gzsl: unseen=0.3028 , seen=0.3904 , h=0.3411
____________________________________________________________________________________________________
episode: 400 loss tensor(2.9538, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.37385449290960926
gzsl: unseen=0.3091 , seen=0.6359 , h=0.4160
____________________________________________________________________________________________________
episode: 600 loss tensor(2.9032, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num:

episode: 5600 loss tensor(1.9507, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6528819367883744
gzsl: unseen=0.5095 , seen=0.7463 , h=0.6055
____________________________________________________________________________________________________
episode: 5800 loss tensor(2.3037, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6457662641456092
gzsl: unseen=0.4992 , seen=0.7463 , h=0.5983
____________________________________________________________________________________________________
episode: 6000 loss tensor(1.9639, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6496830317158968
gzsl: unseen=0.5242 , seen=0.7474 , h=0.6162
____________________________________________________________________________________________________
episode: 6200 loss tensor(1.9569, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
clas

episode: 11200 loss tensor(2.0164, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6624556532639181
gzsl: unseen=0.5372 , seen=0.7818 , h=0.6368
____________________________________________________________________________________________________
episode: 11400 loss tensor(2.0533, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6553097591187828
gzsl: unseen=0.5063 , seen=0.7887 , h=0.6167
____________________________________________________________________________________________________
episode: 11600 loss tensor(2.0423, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6533328474506517
gzsl: unseen=0.5223 , seen=0.7857 , h=0.6275
____________________________________________________________________________________________________
episode: 11800 loss tensor(1.8976, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50


episode: 16800 loss tensor(1.6674, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6732120622015818
gzsl: unseen=0.5427 , seen=0.7953 , h=0.6451
____________________________________________________________________________________________________
episode: 17000 loss tensor(1.6067, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6750611794436817
gzsl: unseen=0.5290 , seen=0.7927 , h=0.6345
____________________________________________________________________________________________________
episode: 17200 loss tensor(2.8439, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6534917641793208
gzsl: unseen=0.4931 , seen=0.8096 , h=0.6129
____________________________________________________________________________________________________
episode: 17400 loss tensor(1.7563, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50


episode: 22400 loss tensor(1.4668, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6461268468182688
gzsl: unseen=0.5006 , seen=0.8079 , h=0.6181
____________________________________________________________________________________________________
episode: 22600 loss tensor(1.8641, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.671134870641126
gzsl: unseen=0.5231 , seen=0.8048 , h=0.6341
____________________________________________________________________________________________________
episode: 22800 loss tensor(2.6038, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6733675975950806
gzsl: unseen=0.5174 , seen=0.8032 , h=0.6294
____________________________________________________________________________________________________
episode: 23000 loss tensor(2.2459, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
c

...

episode: 72800 loss tensor(1.4701, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6429605111894723
gzsl: unseen=0.4986 , seen=0.8159 , h=0.6189
____________________________________________________________________________________________________
episode: 73000 loss tensor(1.4416, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6437954500512582
gzsl: unseen=0.5024 , seen=0.8109 , h=0.6204
____________________________________________________________________________________________________
episode: 73200 loss tensor(2.6499, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.642660391228538
gzsl: unseen=0.4999 , seen=0.8112 , h=0.6186
____________________________________________________________________________________________________
episode: 73400 loss tensor(1.9853, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
c

episode: 78400 loss tensor(0.7490, device='cuda:0')
loss_zero_number=  1
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6434199613119
gzsl: unseen=0.4935 , seen=0.8145 , h=0.6146
____________________________________________________________________________________________________
episode: 78600 loss tensor(0.5920, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6466255659155669
gzsl: unseen=0.4934 , seen=0.8166 , h=0.6151
____________________________________________________________________________________________________
episode: 78800 loss tensor(2.4805, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6432101855722506
gzsl: unseen=0.4912 , seen=0.8171 , h=0.6135
____________________________________________________________________________________________________
episode: 79000 loss tensor(1.3050, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
cla

episode: 84000 loss tensor(1.9829, device='cuda:0')
loss_zero_number=  2
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6408356444408401
gzsl: unseen=0.5049 , seen=0.8110 , h=0.6224
____________________________________________________________________________________________________
episode: 84200 loss tensor(1.2131, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6432512827276049
gzsl: unseen=0.5044 , seen=0.8130 , h=0.6226
____________________________________________________________________________________________________
episode: 84400 loss tensor(1.0627, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6439287136516891
gzsl: unseen=0.5059 , seen=0.8119 , h=0.6234
____________________________________________________________________________________________________
episode: 84600 loss tensor(2.1800, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50


episode: 89600 loss tensor(1.5944, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.642939545579683
gzsl: unseen=0.4962 , seen=0.8148 , h=0.6168
____________________________________________________________________________________________________
episode: 89800 loss tensor(1.4590, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6403350567899048
gzsl: unseen=0.4952 , seen=0.8157 , h=0.6163
____________________________________________________________________________________________________
episode: 90000 loss tensor(1.8622, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6405608534629286
gzsl: unseen=0.4949 , seen=0.8151 , h=0.6159
____________________________________________________________________________________________________
episode: 90200 loss tensor(1.9694, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
c

episode: 95200 loss tensor(0.7898, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6427067179227672
gzsl: unseen=0.4992 , seen=0.8145 , h=0.6190
____________________________________________________________________________________________________
episode: 95400 loss tensor(1.8375, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6406209964268843
gzsl: unseen=0.4979 , seen=0.8141 , h=0.6179
____________________________________________________________________________________________________
episode: 95600 loss tensor(2.1343, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6445364056258048
gzsl: unseen=0.4998 , seen=0.8136 , h=0.6192
____________________________________________________________________________________________________
episode: 95800 loss tensor(1.5809, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
